# dFBA_simulation
- dFBA: dynamic flux balance analysis

**1. Dependency**
- MATLAB v9.6 (R2019a)
- The COnstraint-Based Reconstruction and Analysis Toolbox v3.0.4

In [ ]:
% Set solver ('GLPK')
solverOK = changeCobraSolver('glpk')

In [ ]:
% Load GEMs
model = readCbModel('input/iSH1474.xml','fileType','SBML')

In [ ]:
% Set glucose uptake rate to 6.5 mmol/gDCW/h
model = changeRxnBounds(model,'EX_glc__D_e',-6.5,'l')

In [ ]:
% Run dFBA
% See link for details (https://opencobra.github.io/cobratoolbox/latest/modules/analysis/dynamicFBA/index.html)

substrateRxns = {'EX_glc__D_e'} % Exchange reaction of substrate
initConcentrations = [10] % Initial concentration (mM)
initBiomass = 0.01 % Initial biomass (g/L) (must be non zero)
timeStep = 0.1 % Time step size
nSteps = 1000 % Maximum number of time steps
plotRxns = {'EX_glc__D_e', 'EX_ac_e'} % Reactions to be plotted (Default = {EX_glc(e)’, ‘EX_ac(e)’, ‘EX_for(e)'})
[concentrationMatrix, excRxnNames, timeVec, biomassVec] = dynamicFBA(model, substrateRxns, initConcentrations, initBiomass, timeStep, nSteps, plotRxns)

In [ ]:
% Trim dFBA result
glc_find = strcmp(excRxnNames, 'EX_glc__D_e')
[numRows, numCols] = size(excRxnNames)
for r = 1:numRows
        if glc_find(r,1) == 1
            glc_ind = r
        end
end

glc = concentrationMatrix (glc_ind, :)

dfba_matrix = [timeVec; glc; biomassVec]
dfba_matrix = dfba_matrix'

In [6]:
% output dFBA result
VarNames = {'Time_h', 'Glucose_mM', 'Biomass_gL'};
dFBA_result = table(dfba_matrix(:,1),dfba_matrix(:,2),dfba_matrix(:,3), 'VariableNames',VarNames)
writetable(dFBA_result,'output/dFBA_result.txt','Delimiter','\t')


dFBA_result =

  72x3 table

    Time_h    Glucose_mM    Biomass_gL
    ______    __________    __________

       0            10           0.01 
     0.1        9.9928       0.010663 
     0.2        9.9852       0.011371 
     0.3        9.9771       0.012125 
     0.4        9.9684       0.012929 
     0.5        9.9591       0.013787 
     0.6        9.9493       0.014702 
     0.7        9.9387       0.015677 
     0.8        9.9275       0.016717 
     0.9        9.9155       0.017826 
       1        9.9028       0.019009 
     1.1        9.8892        0.02027 
     1.2        9.8747       0.021614 
     1.3        9.8592       0.023048 
     1.4        9.8427       0.024577 
     1.5        9.8251       0.026208 
     1.6        9.8063       0.027946 
     1.7        9.7863         0.0298 
     1.8         9.765       0.031777 
     1.9        9.7423       0.033885 
       2         9.718       0.036133 
     2.1        9.6921        0.03853 
     2.2        9.6645       0.04

In [7]:
% Load batch culture data & dFBA result
batch_culture = readtable('input/batch_culture.txt')
dFBA_result = readtable('output/dFBA_result.txt')


batch_culture =

  12x3 table

    Time_h    Glucose_mM    Biomass_gL
    ______    __________    __________

       0         10.01        0.0033  
     0.7        10.431        0.0059  
     1.5         10.01        0.0199  
     2.5        9.2147        0.0337  
     3.5        7.8738         0.046  
     4.5        6.8291        0.1118  
     5.5        4.9893         0.202  
       6        2.7909        0.4781  
     6.5           NaN        0.5996  
     7.5           NaN        0.9712  
     8.2        0.1403        1.0844  
      10        0.1403        1.0862  


dFBA_result =

  72x3 table

    Time_h    Glucose_mM    Biomass_gL
    ______    __________    __________

       0            10           0.01 
     0.1        9.9928       0.010663 
     0.2        9.9852       0.011371 
     0.3        9.9771       0.012125 
     0.4        9.9684       0.012929 
     0.5        9.9591       0.013787 
     0.6        9.9493       0.014702 
     0.7        9.9387       0.015677 

In [ ]:
% Get correlation coefficient value of glucose and biomass (r) - 1. Trimming data

% Convert table to array form
bc_array = table2array(batch_culture)
dfba_array = table2array(dFBA_result)

% Extract time data from experimental and dFBA simulation results
bc_time = bc_array(:, 1)
dfba_time = dfba_array(:, 1)

% Get common time and index from both time data
[time_common, bt_i, dt_i] = intersect(bc_time, dfba_time)

% From each result, extract corresponding glucose concentration and biomass data using common time index data
bc_glc = []
bc_biomass = []
for i = bt_i
    bc_glc = [bc_glc, bc_array(i, 2)]
    bc_biomass = [bc_biomass, bc_array(i, 3)]
end

dfba_glc = []
dfba_biomass = []
for i = dt_i
    dfba_glc = [dfba_glc, dfba_array(i, 2)]
    dfba_biomass = [dfba_biomass, dfba_array(i, 3)]
end

% Check the presence of NaN among the extracted glucose concentration and biomass data
bc_glc_tf = isnan(bc_glc)
bc_biomass_tf = isnan(bc_biomass)

dfba_glc_tf = isnan(dfba_glc)
dfba_biomass_tf = isnan(dfba_biomass)

% Remove the index where NaN exists in either result.
glc_tf = bc_glc_tf + dfba_glc_tf
biomass_tf = bc_biomass_tf + dfba_biomass_tf

[bt_row, bt_col] = size(bt_i)
[dt_row, dt_col] = size(dt_i)

bc_glc_filter = []
bc_biomass_filter = []
dfba_glc_filter = []
dfba_biomass_filter = []
for r = 1:bt_row
    if glc_tf(r, 1) == 0
        bc_glc_filter = [bc_glc_filter, bc_glc(r, 1)]
        dfba_glc_filter = [dfba_glc_filter, dfba_glc(r, 1)]
    end
    if biomass_tf(r, 1) == 0
        bc_biomass_filter = [bc_biomass_filter, bc_biomass(r, 1)]
        dfba_biomass_filter = [dfba_biomass_filter, dfba_biomass(r, 1)]
    end
end

In [9]:
% Get correlation coefficient value of glucose and biomass (r) - 2. Correlation coefficient result

% Calculate the correlation coefficient value from the concentration glucose and biomass data of the both trimmed results
r_glc = corrcoef(bc_glc_filter,dfba_glc_filter)
r_biomass = corrcoef(bc_biomass_filter, dfba_biomass_filter)

r_glc_value = r_glc(1,2)
r_biomass_value = r_biomass(1,2)


r_glc =

    1.0000    0.9878
    0.9878    1.0000


r_biomass =

    1.0000    0.9805
    0.9805    1.0000


r_glc_value =

    0.9878


r_biomass_value =

    0.9805


